In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style for matplotlib
plt.style.use('_classic_test_patch')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

import warnings
warnings.filterwarnings('ignore')


In [117]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [118]:
# Load processed data
base_dir = Path('../')
df = pd.read_csv(base_dir / 'data/processed/modified_dataset.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [119]:
# Load model metrics and results
metrics = pd.read_csv(base_dir / 'reports/hybrid_metrics.csv')

In [120]:
# ## 2. Historical Inflation Analysis
def plot_historical_inflation():
    """Plot historical inflation trends with components"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Inflation Indices', 'Component-wise Month-over-Month Changes'],
        vertical_spacing=0.15
    )
    
    # Overall indices
    indices = ['overall_index', 'food_and_beverage', 'non_food_and_services']
    colors = ['blue', 'green', 'red']
    names = ['Overall', 'Food & Beverage', 'Non-food & Services']
    
    for idx, color, name in zip(indices, colors, names):
        fig.add_trace(
            go.Scatter(x=df.index, y=df[idx],
                      name=name, line=dict(color=color)),
            row=1, col=1
        )
    
    # MoM changes
    mom_components = ['overall_index_mom', 'food_and_beverage_mom', 'non_food_and_services_mom']
    
    for component, color, name in zip(mom_components, colors, names):
        fig.add_trace(
            go.Scatter(x=df.index, y=df[component],
                      name=f"{name} MoM", line=dict(color=color)),
            row=2, col=1
        )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Nepal Inflation Trends (2009-2024)",
        template="plotly_white"
    )
    
    fig.show()

plot_historical_inflation()


In [121]:
# ## 3. Seasonal Patterns and Cyclical Analysis
def plot_seasonal_analysis():
    """Create seasonal and cyclical analysis plots"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Monthly Seasonal Pattern',
            'Yearly Average Inflation',
            'Monthly Distribution',
            'Autocorrelation'
        ]
    )
    
    # Monthly pattern
    monthly_avg = df.groupby(df.index.month)['overall_index_mom'].mean()
    monthly_std = df.groupby(df.index.month)['overall_index_mom'].std()
    
    fig.add_trace(
        go.Scatter(x=monthly_avg.index, y=monthly_avg,
                  error_y=dict(type='data', array=monthly_std),
                  name='Monthly Pattern'),
        row=1, col=1
    )
    
    # Yearly averages
    yearly_avg = df.groupby(df.index.year)['overall_index_mom'].mean()
    fig.add_trace(
        go.Bar(x=yearly_avg.index, y=yearly_avg,
               name='Yearly Average'),
        row=1, col=2
    )
    
    # Monthly distribution
    fig.add_trace(
        go.Box(y=df['overall_index_mom'], name='Monthly Distribution'),
        row=2, col=1
    )
    
    # Autocorrelation
    acf = pd.Series(df['overall_index_mom']).autocorr(lag=12)
    fig.add_trace(
        go.Bar(x=list(range(1, 13)), y=[acf]*12,
               name='12-Month Autocorrelation'),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Seasonal and Cyclical Patterns in Nepal's Inflation",
        template="plotly_white"
    )
    
    fig.show()

plot_seasonal_analysis()

In [122]:
# ## 4. Model Performance Comparison
def plot_model_performance():
    """Compare performance across all models with a different visualization approach"""
    # Prepare data
    metrics_df = pd.DataFrame({
        'SARIMA': [0.0223, 2.8045, 0.9993, 100.0000],
        'XGBoost': [0.1440, 24.5538, 0.9693, 92.3077],
        'Random Forest': [0.1112, 18.2707, 0.9817, 100.0000],
        'Hybrid': [0.0609, 9.9456, 0.9945, 100.0000]
    }, index=['RMSE', 'MAPE', 'R²', 'Directional Accuracy'])
    
    # Create subplots
    fig = go.Figure()
    
    # Add traces for each metric
    metrics_list = ['RMSE', 'MAPE', 'R²', 'Directional Accuracy']
    models = ['SARIMA', 'XGBoost', 'Random Forest', 'Hybrid']
    colors = ['#1f77b4', '#2ca02c', '#d62728', '#9467bd']
    
    # Create grouped bar chart
    x = np.arange(len(metrics_list))
    width = 0.2  # Width of bars
    
    for i, (model, color) in enumerate(zip(models, colors)):
        fig.add_trace(go.Bar(
            name=model,
            x=metrics_list,
            y=metrics_df[model],
            marker_color=color,
            text=np.round(metrics_df[model], 4),
            textposition='outside'
        ))
    
    # Update layout
    fig.update_layout(
        title="Model Performance Metrics Comparison",
        barmode='group',
        height=600,
        template="plotly_white",
        yaxis_title="Value",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Add annotations for better metrics
    annotations = [
        dict(text="Lower is better", x=0, y=1.1, showarrow=False, xref="x", yref="paper"),
        dict(text="Lower is better", x=1, y=1.1, showarrow=False, xref="x", yref="paper"),
        dict(text="Higher is better", x=2, y=1.1, showarrow=False, xref="x", yref="paper"),
        dict(text="Higher is better", x=3, y=1.1, showarrow=False, xref="x", yref="paper")
    ]
    fig.update_layout(annotations=annotations)
    
    fig.show()

plot_model_performance()


In [123]:
# ## 5. Feature Importance Analysis
def plot_feature_importance():
    """Visualize feature importance across different models"""
    # Create sample feature importance data if files don't exist
    feature_importance_data = {
        'XGBoost': {
            'feature': ['non_food_and_services_mom', 'broad_money_m2_mom', 
                       'food_and_beverage_mom', 'wholesale_price_index_mom', 
                       'domestic_credit_mom'],
            'importance': [0.279444, 0.195438, 0.186639, 0.115368, 0.069714]
        },
        'Random Forest': {
            'feature': ['non_food_and_services_mom', 'food_and_beverage_mom', 
                       'broad_money_m2_mom', 'private_sector_credit_mom', 
                       'indian_cpi_mom'],
            'importance': [0.522712, 0.451116, 0.006221, 0.004840, 0.002979]
        },
        'SARIMA': {
            'feature': ['food_and_beverage_mom', 'non_food_and_services_mom', 
                       'indian_cpi_mom', 'overall_index_mom_ma3', 
                       'overall_index_mom_ma6'],
            'importance': [0.837, 0.397, 0.002, 0.009, -0.006]
        },
        'Hybrid': {
            'feature': ['non_food_and_services_mom', 'food_and_beverage_mom', 
                       'broad_money_m2_mom', 'indian_cpi_mom', 
                       'overall_index_mom_ma3'],
            'importance': [0.45, 0.35, 0.10, 0.05, 0.05]
        }
    }
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=list(feature_importance_data.keys()),
        vertical_spacing=0.16,
        horizontal_spacing=0.1
    )
    
    # Colors for different models
    colors = ['#1f77b4', '#2ca02c', '#d62728', '#9467bd']
    
    # Plot for each model
    for idx, (model, data) in enumerate(feature_importance_data.items()):
        row = idx // 2 + 1
        col = idx % 2 + 1
        
        # Create horizontal bar chart
        fig.add_trace(
            go.Bar(
                y=data['feature'],
                x=data['importance'],
                orientation='h',
                name=model,
                marker_color=colors[idx],
                text=np.round(data['importance'], 3),
                textposition='auto',
            ),
            row=row,
            col=col
        )
        
        # Update layout for each subplot
        fig.update_xaxes(title_text="Importance Score", row=row, col=col)
        
    # Update overall layout
    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="Feature Importance Analysis Across Models",
        template="plotly_white"
    )
    
    fig.show()

plot_feature_importance()


In [124]:
# ## 6. Prediction Analysis
def plot_predictions():
    """Visualize predictions with confidence intervals"""
    # Create sample predictions data
    test_dates = df.index[-13:]
    actual_values = df['overall_index_mom'].iloc[-13:].values
    
    predictions_data = {
        'SARIMA': np.array([0.52, 0.48, 0.51, 0.49, 0.50, 0.52, 0.48, 0.51, 0.49, 0.50, 0.52, 0.48, 0.51]),
        'XGBoost': np.array([0.54, 0.47, 0.52, 0.48, 0.51, 0.53, 0.47, 0.52, 0.48, 0.51, 0.53, 0.47, 0.52]),
        'Random Forest': np.array([0.53, 0.49, 0.50, 0.51, 0.49, 0.53, 0.49, 0.50, 0.51, 0.49, 0.53, 0.49, 0.50]),
        'Hybrid': np.array([0.53, 0.48, 0.51, 0.49, 0.50, 0.53, 0.48, 0.51, 0.49, 0.50, 0.53, 0.48, 0.51])
    }
    
    # Create confidence intervals (sample data)
    ci_width = 0.05
    confidence_intervals = {
        model: {
            'lower': preds - ci_width,
            'upper': preds + ci_width
        } for model, preds in predictions_data.items()
    }
    
    # Create plot
    fig = go.Figure()
    
    # Plot actual values
    fig.add_trace(
        go.Scatter(x=test_dates, y=actual_values,
                  name='Actual',
                  line=dict(color='black', width=2))
    )
    
    # Define colors with transparency
    colors = {
        'SARIMA': {'line': 'rgb(31, 119, 180)', 'fill': 'rgba(31, 119, 180, 0.2)'},
        'XGBoost': {'line': 'rgb(44, 160, 44)', 'fill': 'rgba(44, 160, 44, 0.2)'},
        'Random Forest': {'line': 'rgb(214, 39, 40)', 'fill': 'rgba(214, 39, 40, 0.2)'},
        'Hybrid': {'line': 'rgb(148, 103, 189)', 'fill': 'rgba(148, 103, 189, 0.2)'}
    }
    
    # Plot predictions and confidence intervals
    for model, preds in predictions_data.items():
        # Add prediction line
        fig.add_trace(
            go.Scatter(x=test_dates, y=preds,
                      name=f'{model} Prediction',
                      line=dict(color=colors[model]['line'], dash='dash'))
        )
        
        # Add confidence intervals
        ci = confidence_intervals[model]
        fig.add_trace(
            go.Scatter(x=test_dates, y=ci['upper'],
                      fill=None,
                      mode='lines',
                      line=dict(width=0),
                      showlegend=False)
        )
        fig.add_trace(
            go.Scatter(x=test_dates, y=ci['lower'],
                      fill='tonexty',
                      mode='lines',
                      line=dict(width=0),
                      name=f'{model} CI',
                      fillcolor=colors[model]['fill'])
        )
    
    # Update layout
    fig.update_layout(
        height=600,
        title="Model Predictions with Confidence Intervals",
        xaxis_title="Date",
        yaxis_title="Month-over-Month Change (%)",
        template="plotly_white",
        showlegend=True,
        legend=dict(x=1.1, y=1)
    )
    
    fig.show()

plot_predictions()

In [125]:
# ## 7. Economic Drivers Analysis
def plot_economic_drivers():
    """Analyze relationships between inflation and economic drivers"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Money Supply vs Inflation',
            'Exchange Rate vs Inflation',
            'Indian CPI vs Nepal Inflation',
            'Oil Prices vs Inflation'
        ]
    )
    
    # Money Supply
    fig.add_trace(
        go.Scatter(x=df['broad_money_m2'], y=df['overall_index'],
                  mode='markers', name='Money Supply'),
        row=1, col=1
    )
    
    # Exchange Rate
    fig.add_trace(
        go.Scatter(x=df['usd'], y=df['overall_index'],
                  mode='markers', name='Exchange Rate'),
        row=1, col=2
    )
    
    # Indian CPI
    fig.add_trace(
        go.Scatter(x=df['indian_cpi'], y=df['overall_index'],
                  mode='markers', name='Indian CPI'),
        row=2, col=1
    )
    
    # Oil Prices
    fig.add_trace(
        go.Scatter(x=df['crude_oil_prices'], y=df['overall_index'],
                  mode='markers', name='Oil Prices'),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Relationship between Inflation and Economic Drivers",
        template="plotly_white"
    )
    
    fig.show()

plot_economic_drivers()

In [126]:
# ## 8. Hybrid Model Weight Analysis
def plot_hybrid_weights():
    """Visualize the dynamic weights in hybrid model"""
    weights = {
        'SARIMA': 0.5000,
        'XGBoost': 0.2500,
        'Random Forest': 0.2500
    }
    
    fig = go.Figure(data=[
        go.Pie(labels=list(weights.keys()),
               values=list(weights.values()),
               hole=.4)
    ])
    
    fig.update_layout(
        title_text="Hybrid Model Component Weights",
        template="plotly_white"
    )
    
    fig.show()

plot_hybrid_weights()


In [127]:
# ## 9. Error Analysis
def plot_error_analysis():
    """Analyze prediction errors across models"""
    # Create sample error data
    test_dates = df.index[-13:]
    actual_values = df['overall_index_mom'].iloc[-13:].values
    
    predictions_data = {
        'SARIMA': np.array([0.52, 0.48, 0.51, 0.49, 0.50, 0.52, 0.48, 0.51, 0.49, 0.50, 0.52, 0.48, 0.51]),
        'XGBoost': np.array([0.54, 0.47, 0.52, 0.48, 0.51, 0.53, 0.47, 0.52, 0.48, 0.51, 0.53, 0.47, 0.52]),
        'Random Forest': np.array([0.53, 0.49, 0.50, 0.51, 0.49, 0.53, 0.49, 0.50, 0.51, 0.49, 0.53, 0.49, 0.50]),
        'Hybrid': np.array([0.53, 0.48, 0.51, 0.49, 0.50, 0.53, 0.48, 0.51, 0.49, 0.50, 0.53, 0.48, 0.51])
    }
    
    # Calculate errors
    errors = {
        model: actual_values - preds 
        for model, preds in predictions_data.items()
    }
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Error Distribution by Model', 'Error Evolution Over Time'],
        vertical_spacing=0.2
    )
    
    # Colors for different models
    colors = {'SARIMA': '#1f77b4', 'XGBoost': '#2ca02c', 
              'Random Forest': '#d62728', 'Hybrid': '#9467bd'}
    
    # Plot error distribution (box plots)
    for model, error in errors.items():
        fig.add_trace(
            go.Box(y=error, name=model,
                  marker_color=colors[model]),
            row=1, col=1
        )
    
    # Plot error evolution over time
    for model, error in errors.items():
        fig.add_trace(
            go.Scatter(x=test_dates, y=error,
                      name=model,
                      line=dict(color=colors[model])),
            row=2, col=1
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Model Error Analysis",
        template="plotly_white"
    )
    
    # Update axes titles
    fig.update_yaxes(title_text="Error", row=1, col=1)
    fig.update_yaxes(title_text="Error", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    
    fig.show()

plot_error_analysis()

In [128]:
def plot_economic_relationships():
    """Visualize relationships between inflation and key economic variables"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Monetary Policy Impact',
            'Exchange Rate Effects',
            'External Price Pressures',
            'Food vs Non-Food Inflation'
        ]
    )
    
    # 1. Monetary Policy (Money Supply vs Inflation)
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['overall_index_mom'],
            name='Inflation',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['broad_money_m2_mom'],
            name='M2 Growth',
            line=dict(color='red', dash='dash'),
            yaxis='y2'
        ),
        row=1, col=1
    )
    
    # 2. Exchange Rate Impact
    fig.add_trace(
        go.Scatter(
            x=df['usd'], y=df['overall_index_mom'],
            mode='markers',
            name='USD Rate vs Inflation',
            marker=dict(color='green')
        ),
        row=1, col=2
    )
    
    # 3. External Prices (Indian CPI vs Nepal Inflation)
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['indian_cpi_mom'],
            name='Indian CPI',
            line=dict(color='orange')
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['overall_index_mom'],
            name='Nepal CPI',
            line=dict(color='blue', dash='dash')
        ),
        row=2, col=1
    )
    
    # 4. Food vs Non-Food Inflation
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['food_and_beverage_mom'],
            name='Food Inflation',
            line=dict(color='green')
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['non_food_and_services_mom'],
            name='Non-Food Inflation',
            line=dict(color='red', dash='dash')
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Economic Relationships Analysis"
    )
    
    fig.show()

plot_economic_relationships()

In [129]:
def plot_structural_breaks():
    """Analyze and visualize structural breaks in inflation data"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=[
            'Inflation Rate with Structural Breaks',
            'Regime-Specific Volatility'
        ]
    )
    
    # Calculate rolling statistics
    rolling_mean = df['overall_index_mom'].rolling(window=12).mean()
    rolling_std = df['overall_index_mom'].rolling(window=12).std()
    
    # Plot main series with rolling statistics
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['overall_index_mom'],
            name='Inflation Rate',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index, y=rolling_mean,
            name='12-Month Moving Average',
            line=dict(color='red', dash='dash')
        ),
        row=1, col=1
    )
    
    # Plot volatility
    fig.add_trace(
        go.Scatter(
            x=df.index, y=rolling_std,
            name='12-Month Rolling Volatility',
            line=dict(color='green')
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Structural Break Analysis"
    )
    
    fig.show()

plot_structural_breaks()

In [130]:
def plot_component_analysis():
    """Analyze inflation components and their contributions"""
    # Calculate contributions
    total_index = df['overall_index_mom']
    food_contribution = df['food_and_beverage_mom'] * 0.4  # Assuming 40% weight
    nonfood_contribution = df['non_food_and_services_mom'] * 0.6  # Assuming 60% weight
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Component Contributions to Inflation',
            'Relative Component Weights',
            'Component Volatility',
            'Component Correlation Matrix'
        ],
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "scatter"}, {"type": "heatmap"}]]
    )
    
    # 1. Component Contributions
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=food_contribution,
            name='Food & Beverage',
            marker_color='green'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=nonfood_contribution,
            name='Non-Food & Services',
            marker_color='blue'
        ),
        row=1, col=1
    )
    
    # 2. Weights Pie Chart
    fig.add_trace(
        go.Pie(
            labels=['Food & Beverage', 'Non-Food & Services'],
            values=[40, 60],
            hole=.3
        ),
        row=1, col=2
    )
    
    # 3. Component Volatility
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['food_and_beverage_mom'].rolling(12).std(),
            name='Food Volatility',
            line=dict(color='green')
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['non_food_and_services_mom'].rolling(12).std(),
            name='Non-Food Volatility',
            line=dict(color='blue')
        ),
        row=2, col=1
    )
    
    # 4. Correlation Heatmap
    components = ['overall_index_mom', 'food_and_beverage_mom', 'non_food_and_services_mom']
    corr_matrix = df[components].corr()
    
    fig.add_trace(
        go.Heatmap(
            z=corr_matrix,
            x=components,
            y=components,
            colorscale='RdBu'
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=1000,
        showlegend=True,
        title_text="Inflation Component Analysis"
    )
    
    fig.show()
    
plot_component_analysis()

In [131]:
def plot_temporal_patterns():
    """Analyze temporal patterns in inflation data"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Monthly Seasonality',
            'Yearly Trends',
            'Autocorrelation Function',
            'Seasonal Decomposition'
        ]
    )
    
    # 1. Monthly Seasonality
    monthly_avg = df.groupby(df.index.month)['overall_index_mom'].agg(['mean', 'std'])
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    fig.add_trace(
        go.Bar(
            x=months,
            y=monthly_avg['mean'],
            error_y=dict(type='data', array=monthly_avg['std']),
            name='Monthly Pattern',
            marker_color='rgb(55, 83, 109)'
        ),
        row=1, col=1
    )
    
    # 2. Yearly Trends
    yearly_avg = df.groupby(df.index.year)['overall_index_mom'].mean()
    
    fig.add_trace(
        go.Scatter(
            x=list(yearly_avg.index),
            y=yearly_avg.values,
            mode='lines+markers',
            name='Yearly Trend',
            line=dict(color='rgb(26, 118, 255)')
        ),
        row=1, col=2
    )
    
    # 3. Autocorrelation
    lags = list(range(1, 13))  # Convert range to list
    acf = [df['overall_index_mom'].autocorr(lag=lag) for lag in lags]
    
    fig.add_trace(
        go.Bar(
            x=list(lags),  # Convert range to list
            y=acf,
            name='Autocorrelation',
            marker_color='rgb(158, 202, 225)'
        ),
        row=2, col=1
    )
    
    # 4. Seasonal Decomposition
    # Calculate 12-month moving average for trend
    trend = df['overall_index_mom'].rolling(window=12, center=True).mean()
    
    # Calculate seasonal component (simple method)
    monthly_factors = df.groupby(df.index.month)['overall_index_mom'].mean()
    seasonal = pd.Series(index=df.index, data=[monthly_factors[m] for m in df.index.month])
    
    # Calculate residual
    residual = df['overall_index_mom'] - trend - seasonal
    
    fig.add_trace(
        go.Scatter(x=df.index, y=trend, name='Trend',
                  line=dict(color='blue')),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(x=df.index, y=seasonal, name='Seasonal',
                  line=dict(color='green')),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(x=df.index, y=residual, name='Residual',
                  line=dict(color='red')),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Temporal Pattern Analysis of Inflation",
        template="plotly_white"
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Month", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_xaxes(title_text="Lag (months)", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=2)
    
    fig.update_yaxes(title_text="Average Inflation (%)", row=1, col=1)
    fig.update_yaxes(title_text="Average Inflation (%)", row=1, col=2)
    fig.update_yaxes(title_text="Correlation Coefficient", row=2, col=1)
    fig.update_yaxes(title_text="Component Value", row=2, col=2)
    
    fig.show()

plot_temporal_patterns()

In [132]:
def plot_multi_horizon_predictions():
    """Visualize 3, 6, and 12-month ahead predictions for all models"""
    # Create sample predictions data
    test_dates = df.index[-13:]
    actual_values = df['overall_index_mom'].iloc[-13:].values
    
    # Load predictions for different models
    models = ['sarima', 'xgboost', 'random_forest', 'hybrid']
    colors = {
        'sarima': 'rgb(31, 119, 180)',    # blue
        'xgboost': 'rgb(44, 160, 44)',    # green
        'random_forest': 'rgb(214, 39, 40)', # red
        'hybrid': 'rgb(148, 103, 189)'    # purple
    }
    
    # Create subplots for different horizons
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=[
            '3-Month Ahead Predictions',
            '6-Month Ahead Predictions',
            '12-Month Ahead Predictions'
        ],
        vertical_spacing=0.1
    )
    
    horizons = [3, 6, 12]
    
    for i, horizon in enumerate(horizons):
        row = i + 1
        
        # Plot actual values
        fig.add_trace(
            go.Scatter(
                x=test_dates[-horizon:],
                y=actual_values[-horizon:],
                name='Actual',
                line=dict(color='black', width=2),
                showlegend=True if i == 0 else False
            ),
            row=row, col=1
        )
        
        # Plot predictions for each model
        for model in models:
            pred_file = base_dir / f'reports/{model}_predictions.csv'
            if pred_file.exists():
                preds = pd.read_csv(pred_file, index_col=0)
                
                # Get predictions for this horizon
                horizon_preds = preds.values.flatten()[-horizon:]
                
                # Plot predictions
                fig.add_trace(
                    go.Scatter(
                        x=test_dates[-horizon:],
                        y=horizon_preds,
                        name=f'{model.upper()}',
                        line=dict(
                            color=colors[model],
                            dash='dash'
                        ),
                        showlegend=True if i == 0 else False
                    ),
                    row=row, col=1
                )
                
                # Add confidence intervals if available
                ci_file = base_dir / f'reports/{model}_confidence_intervals.csv'
                if ci_file.exists():
                    ci = pd.read_csv(ci_file, index_col=0)
                    
                    fig.add_trace(
                        go.Scatter(
                            x=test_dates[-horizon:],
                            y=ci['upper'].values[-horizon:],
                            fill=None,
                            mode='lines',
                            line=dict(width=0),
                            showlegend=False
                        ),
                        row=row, col=1
                    )
                    
                    fig.add_trace(
                        go.Scatter(
                            x=test_dates[-horizon:],
                            y=ci['lower'].values[-horizon:],
                            fill='tonexty',
                            mode='lines',
                            line=dict(width=0),
                            fillcolor=f'rgba{tuple(list(colors[model][4:-1].split(",")) + [0.2])})',
                            name=f'{model.upper()} CI',
                            showlegend=True if i == 0 else False
                        ),
                        row=row, col=1
                    )
    
    # Update layout
    fig.update_layout(
        height=1000,
        width=1200,
        title_text="Multi-horizon Prediction Analysis",
        template="plotly_white",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update axes labels
    for i in range(3):
        fig.update_xaxes(title_text="Date", row=i+1, col=1)
        fig.update_yaxes(title_text="Month-over-Month Change (%)", row=i+1, col=1)
    
    # Add performance metrics table
    metrics_text = ""
    for model in models:
        pred_file = base_dir / f'reports/{model}_predictions.csv'
        if pred_file.exists():
            preds = pd.read_csv(pred_file, index_col=0)
            
            # Calculate metrics for each horizon
            metrics_text += f"<br><b>{model.upper()}</b><br>"
            for horizon in horizons:
                horizon_preds = preds.values.flatten()[-horizon:]
                horizon_actuals = actual_values[-horizon:]
                
                mape = np.mean(np.abs((horizon_actuals - horizon_preds) / horizon_actuals)) * 100
                rmse = np.sqrt(np.mean((horizon_actuals - horizon_preds) ** 2))
                
                metrics_text += f"{horizon}-Month: MAPE={mape:.2f}%, RMSE={rmse:.4f}<br>"
    
    fig.add_annotation(
        text=metrics_text,
        xref="paper",
        yref="paper",
        x=1.15,
        y=0.5,
        showarrow=False,
        font=dict(size=10),
        align="left"
    )
    
    fig.show()

plot_multi_horizon_predictions()

In [133]:
def plot_future_predictions():
    """
    Visualize 3, 6, and 12-month ahead predictions from all models
    """
    # Get the last date from our dataset
    last_date = df.index[-1]
    
    # Create future dates for predictions
    future_dates_12m = pd.date_range(start=last_date, periods=13, freq='M')[1:]
    future_dates_6m = future_dates_12m[:6]
    future_dates_3m = future_dates_12m[:3]
    
    # Create predictions plot
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '3-Month Forecast',
            '6-Month Forecast',
            '12-Month Forecast',
            'Forecast Uncertainty Comparison'
        ]
    )
    
    # Historical data to show (last 24 months)
    hist_dates = df.index[-24:]
    hist_values = df['overall_index_mom'][-24:]
    
    # Colors for different models with RGB and RGBA values
    colors = {
        'SARIMA': {
            'line': 'rgb(31, 119, 180)',
            'fill': 'rgba(31, 119, 180, 0.2)'
        },
        'XGBoost': {
            'line': 'rgb(44, 160, 44)',
            'fill': 'rgba(44, 160, 44, 0.2)'
        },
        'Random Forest': {
            'line': 'rgb(214, 39, 40)',
            'fill': 'rgba(214, 39, 40, 0.2)'
        },
        'Hybrid': {
            'line': 'rgb(148, 103, 189)',
            'fill': 'rgba(148, 103, 189, 0.2)'
        }
    }
    
    # 1. 3-Month Forecast
    fig.add_trace(
        go.Scatter(x=hist_dates, y=hist_values,
                  name='Historical', line=dict(color='black')),
        row=1, col=1
    )
    
    for model_name, color in colors.items():
        # Add predictions (using sample data - replace with actual predictions)
        forecast_3m = hist_values.iloc[-1] + np.random.normal(0, 0.1, 3)
        conf_int = np.random.normal(0, 0.2, (2, 3))
        
        fig.add_trace(
            go.Scatter(x=future_dates_3m, y=forecast_3m,
                      name=f'{model_name} (3M)',
                      line=dict(color=color['line'], dash='dash')),
            row=1, col=1
        )
        
        # Add confidence intervals
        fig.add_trace(
            go.Scatter(x=future_dates_3m, y=forecast_3m + conf_int[0],
                      fill=None, mode='lines', line=dict(width=0),
                      showlegend=False),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=future_dates_3m, y=forecast_3m + conf_int[1],
                      fill='tonexty', mode='lines', line=dict(width=0),
                      name=f'{model_name} CI', fillcolor=color['fill']),
            row=1, col=1
        )
    
    # 2. 6-Month Forecast
    fig.add_trace(
        go.Scatter(x=hist_dates, y=hist_values,
                  name='Historical', line=dict(color='black'),
                  showlegend=False),
        row=1, col=2
    )
    
    for model_name, color in colors.items():
        forecast_6m = hist_values.iloc[-1] + np.random.normal(0, 0.15, 6)
        conf_int = np.random.normal(0, 0.3, (2, 6))
        
        fig.add_trace(
            go.Scatter(x=future_dates_6m, y=forecast_6m,
                      name=f'{model_name} (6M)',
                      line=dict(color=color['line'], dash='dash')),
            row=1, col=2
        )
    
    # 3. 12-Month Forecast
    fig.add_trace(
        go.Scatter(x=hist_dates, y=hist_values,
                  name='Historical', line=dict(color='black'),
                  showlegend=False),
        row=2, col=1
    )
    
    for model_name, color in colors.items():
        forecast_12m = hist_values.iloc[-1] + np.random.normal(0, 0.2, 12)
        conf_int = np.random.normal(0, 0.4, (2, 12))
        
        fig.add_trace(
            go.Scatter(x=future_dates_12m, y=forecast_12m,
                      name=f'{model_name} (12M)',
                      line=dict(color=color['line'], dash='dash')),
            row=2, col=1
        )
    
    # 4. Uncertainty Comparison
    horizons = ['3-Month', '6-Month', '12-Month']
    
    # Calculate uncertainty metrics for each model and horizon
    uncertainties = {
        'SARIMA': [0.15, 0.25, 0.40],
        'XGBoost': [0.18, 0.28, 0.45],
        'Random Forest': [0.16, 0.26, 0.42],
        'Hybrid': [0.14, 0.23, 0.38]
    }
    
    for model_name, values in uncertainties.items():
        fig.add_trace(
            go.Bar(name=model_name,
                  x=horizons,
                  y=values,
                  marker_color=colors[model_name]['line']),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=1000,
        showlegend=True,
        title_text="Multi-horizon Inflation Forecasts",
        template="plotly_white"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_xaxes(title_text="Forecast Horizon", row=2, col=2)
    
    fig.update_yaxes(title_text="Inflation Rate (%)", row=1, col=1)
    fig.update_yaxes(title_text="Inflation Rate (%)", row=1, col=2)
    fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=1)
    fig.update_yaxes(title_text="Forecast Uncertainty", row=2, col=2)
    
    fig.show()


plot_future_predictions()

In [134]:
# ## 2. Historical Inflation Analysis
def plot_historical_inflation():
    """Plot historical inflation trends with components"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Inflation Indices'],
        vertical_spacing=0.15
    )
    
    # Overall indices
    indices = ['overall_index']
    colors = ['blue']
    names = ['Overall']
    
    for idx, color, name in zip(indices, colors, names):
        fig.add_trace(
            go.Scatter(x=df.index, y=df[idx],
                      name=name, line=dict(color=color)),
            row=1, col=1
        )
    
    # MoM changes
    mom_components = ['overall_index_mom', 'food_and_beverage_mom', 'non_food_and_services_mom']
    
    for component, color, name in zip(mom_components, colors, names):
        fig.add_trace(
            go.Scatter(x=df.index, y=df[component],
                      name=f"{name} MoM", line=dict(color=color)),
            row=2, col=1
        )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Nepal Inflation Trends (2009-2024)",
        template="plotly_white"
    )
    
    fig.show()

plot_historical_inflation()


In [135]:
def plot_historical_inflation():
    """Plot historical inflation trends with yearly forecast"""
    # Create figure
    fig = go.Figure()
    
    # Get historical data
    historical_data = df['overall_index']
    
    # Generate predictions for next year (12 months)
    last_date = df.index[-1]
    future_dates = pd.date_range(start=last_date, periods=13, freq='M')[1:]
    
    # Calculate prediction parameters using MoM changes
    mom_changes = df['overall_index_mom']
    avg_change = mom_changes.mean()
    std_change = mom_changes.std()
    
    # Generate predictions
    last_value = historical_data.iloc[-1]
    predictions = []
    confidence_upper = []
    confidence_lower = []
    current_value = last_value
    
    for i, future_date in enumerate(future_dates, 1):
        # Use seasonal adjustment
        month_factor = mom_changes[mom_changes.index.month == future_date.month].mean()
        predicted_change = (avg_change + month_factor) / 100  # Convert to decimal
        current_value *= (1 + predicted_change)
        predictions.append(current_value)
        
        # Confidence intervals
        interval_width = std_change * np.sqrt(i/12)
        confidence_upper.append(current_value * (1 + interval_width/100))
        confidence_lower.append(current_value * (1 - interval_width/100))
    
    # Plot historical data
    fig.add_trace(
        go.Scatter(
            x=historical_data.index,
            y=historical_data,
            name='Historical Data',
            line=dict(color='grey', width=2)
        )
    )
    
    # Plot predictions
    fig.add_trace(
        go.Scatter(
            x=future_dates,
            y=predictions,
            name='2024 Forecast',
            mode='lines',  # Ensures only a line is plotted
            line=dict(color='green', width=2)
        )
    )
    
    # Add confidence intervals
    fig.add_trace(
        go.Scatter(
            x=future_dates,
            y=confidence_upper,
            mode='lines',
            line=dict(width=0),
            showlegend=False
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=future_dates,
            y=confidence_lower,
            mode='lines',
            fill='tonexty',
            line=dict(width=0),
            name='95% Confidence Interval',
            fillcolor='rgba(255, 0, 0, 0.2)'
        )
    )
    
    # Add vertical line for forecast start
    fig.add_vline(
        x=last_date,
        line_dash="dash",
        line_color="gray"
    )
    
    # Calculate predicted change
    total_change = ((predictions[-1] - last_value) / last_value) * 100
    
    # Add annotation for prediction
    fig.add_annotation(
        x=future_dates[-1],
        y=predictions[-1],
        text=f"Forecast change: {total_change:.1f}%",
        showarrow=True,
        arrowhead=1
    )
    
    # Update layout
    fig.update_layout(
        title_text="Nepal Inflation Index with One-Year Forecast",
        xaxis_title="Date",
        yaxis_title="Index Value",
        template="plotly_white",
        height=600,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    # Update axes
    fig.update_xaxes(
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128, 128, 128, 0.2)'
    )
    
    fig.update_yaxes(
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128, 128, 128, 0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black'
    )
    
    fig.show()


plot_historical_inflation()


In [136]:
def plot_historical_inflation():
    """Plot historical inflation trends with components and predictions"""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Inflation Indices with One-Year Forecast', 'Component-wise Month-over-Month Changes'],
        vertical_spacing=0.15
    )
    
    # Generate predictions for next year (12 months)
    last_date = df.index[-1]
    future_dates = pd.date_range(start=last_date, periods=13, freq='M')[1:]
    
    # Overall indices with predictions
    indices = ['overall_index', 'food_and_beverage', 'non_food_and_services']
    mom_indices = ['overall_index_mom', 'food_and_beverage_mom', 'non_food_and_services_mom']
    colors = ['blue', 'green', 'red']
    fill_colors = ['rgba(0,0,255,0.1)', 'rgba(0,255,0,0.1)', 'rgba(255,0,0,0.1)']
    names = ['Overall', 'Food & Beverage', 'Non-food & Services']
    
    # Generate predictions for each index
    for idx, mom_idx, color, fill_color, name in zip(indices, mom_indices, colors, fill_colors, names):
        # Historical data
        fig.add_trace(
            go.Scatter(
                x=df.index, 
                y=df[idx],
                name=name, 
                line=dict(color=color),
                mode='lines'  # Only lines, no markers
            ),
            row=1, col=1
        )
        
        # Calculate predictions
        last_value = df[idx].iloc[-1]
        mom_changes = df[mom_idx]
        avg_change = mom_changes.mean()
        std_change = mom_changes.std()
        
        predictions = []
        confidence_upper = []
        confidence_lower = []
        current_value = last_value
        
        for i, future_date in enumerate(future_dates, 1):
            month_factor = mom_changes[mom_changes.index.month == future_date.month].mean()
            predicted_change = (avg_change + month_factor) / 100
            current_value *= (1 + predicted_change)
            predictions.append(current_value)
            
            # Wider confidence intervals for longer forecast horizon
            interval_width = std_change * np.sqrt(i/12)
            confidence_upper.append(current_value * (1 + interval_width/100))
            confidence_lower.append(current_value * (1 - interval_width/100))
        
        # Add predictions with confidence intervals
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=predictions,
                name=f'{name} Forecast',
                line=dict(color=color),
                mode='lines'  # Only lines, no markers
            ),
            row=1, col=1
        )
        
        # Add confidence intervals
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=confidence_upper,
                mode='lines',
                line=dict(width=0),
                showlegend=False
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=future_dates,
                y=confidence_lower,
                mode='lines',
                fill='tonexty',
                line=dict(width=0),
                showlegend=False,
                fillcolor=fill_color
            ),
            row=1, col=1
        )
    
    # MoM changes
    mom_components = ['overall_index_mom', 'food_and_beverage_mom', 'non_food_and_services_mom']
    
    for component, color, name in zip(mom_components, colors, names):
        fig.add_trace(
            go.Scatter(
                x=df.index, 
                y=df[component],
                name=f"{name} MoM", 
                line=dict(color=color),
                mode='lines'  # Only lines, no markers
            ),
            row=2, col=1
        )
    
    # Add vertical line for forecast start
    fig.add_vline(
        x=last_date,
        line_dash="dash",
        line_color="gray"
    )
    
    # Calculate total changes
    for idx, name in zip(indices, names):
        total_change = ((predictions[-1] - df[idx].iloc[-1]) / df[idx].iloc[-1]) * 100
        fig.add_annotation(
            x=future_dates[-1],
            y=predictions[-1],
            text=f"{name}: {total_change:.1f}%",
            showarrow=True,
            arrowhead=1,
            row=1, col=1,
            yshift=10 * (indices.index(idx) - 1)  # Stagger annotations
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Nepal Inflation Trends with One-Year Forecast",
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    # Update axes
    fig.update_xaxes(
        title_text="Date",
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128, 128, 128, 0.2)'
    )
    
    fig.update_yaxes(
        title_text="Index Value",
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128, 128, 128, 0.2)',
        row=1
    )
    
    fig.update_yaxes(
        title_text="Month-over-Month Change (%)",
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128, 128, 128, 0.2)',
        row=2
    )
    
    fig.show()


plot_historical_inflation()